In [8]:
import pandas as pd
import numpy as np

#install dependencies: torch, pandas_path, torchvision, pytorch_lightning, fasttext

from src.hatefuldata import HatefulMemesDataset
from src.multimodel import HatefulMemesModel, LanguageAndVisionConcat

In [11]:
df = pd.read_json('data/train.jsonl', lines=True)
df.head()

,id,img,label,text
0,42953,img/42953.png,0,its their character not their color that matters
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...
2,13894,img/13894.png,0,putting bows on your pet
3,37408,img/37408.png,0,i love everything and everybody! except for sq...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h..."


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8500 entries, 0 to 8499
Data columns (total 4 columns):
id       8500 non-null int64
img      8500 non-null object
label    8500 non-null int64
text     8500 non-null object
dtypes: int64(2), object(2)
memory usage: 265.8+ KB


In [15]:
df['label'].value_counts()

0    5450
1    3050
Name: label, dtype: int64

In [16]:
hparams = {
    
    # Required hparams
    "train_path": 'data/train.jsonl',
    "dev_path": 'data/dev.jsonl',
    "img_dir": 'data/',
    
    # Optional hparams
    "embedding_dim": 150,
    "language_feature_dim": 300,
    "vision_feature_dim": 300,
    "fusion_output_size": 256,
    "output_path": "model-outputs",
    "dev_limit": None,
    "lr": 0.00005,
    "max_epochs": 10,
    "n_gpu": 0,
    "batch_size": 4,
    # allows us to "simulate" having larger batches 
    "accumulate_grad_batches": 16,
    "early_stop_patience": 3,
}

hateful_memes_model = HatefulMemesModel(hparams=hparams)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /Users/benw/.cache/torch/checkpoints/resnet152-b121ed2d.pth


EarlyStopping mode auto is unknown, fallback to auto mode.
EarlyStopping mode set to min for monitoring avg_val_loss.


In [18]:
hateful_memes_model.fit()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type                    | Params
--------------------------------------------------
0 | model | LanguageAndVisionConcat | 58 M  


In [ ]:
# we should only have saved the best checkpoint
checkpoints = list(Path("model-outputs").glob("*.ckpt"))
assert len(checkpoints) == 1

checkpoints

In [ ]:
hateful_memes_model = HatefulMemesModel.load_from_checkpoint(
    checkpoints[0]
)
submission = hateful_memes_model.make_submission_frame(
    test_path
)
submission.head()

In [ ]:
submission.groupby("label").proba.mean()

In [ ]:
submission.label.value_counts()

In [ ]:
submission.to_csv(("model-outputs/submission.csv"), index=True)